In [ ]:
from pathlib import Path
import pandas as pd
import torch
from tqdm import tqdm
from ThermalFold.predict_utils import thermalFold_predictor
from ThermalFold.esm_utils import ESM_embedding,ESMH5Cache
from ThermalFold.data_utils import parse_key_name
from lightning.fabric import seed_everything
import warnings
import shutil
warnings.filterwarnings('ignore')

Type checking is disabled.
Debugging is disabled.


[16:31:30] UFFTYPER: Warning: hybridization set to SP3 for atom 0
[16:31:30] UFFTYPER: Unrecognized charge state for atom: 0
[16:31:30] UFFTYPER: Unrecognized atom type: Mn2+2 (0)
[16:31:30] UFFTYPER: Unrecognized atom type: Fe2+2 (0)
[16:31:30] UFFTYPER: Unrecognized atom type: Co3+3 (0)
[16:31:30] UFFTYPER: Unrecognized atom type: Ni3+2 (0)
[16:31:30] UFFTYPER: Unrecognized charge state for atom: 0
[16:31:30] UFFTYPER: Unrecognized atom type: Zn+2 (0)
[16:31:30] UFFTYPER: Unrecognized atom type: Ca+2 (0)


## Temperature ramping examples

In [4]:
cfg_fname = '../weight/model_conf.yaml'
weight_path  = '../weight/model_weight.pt'

df = pd.read_csv("seqs/examples_1.csv")

seed_everything(42)
temp_lst = list(range(273,474,5))

if Path("ThermalFold_prediction").is_dir(): shutil.rmtree('ThermalFold_prediction')
with ESM_embedding('esm2_650M',device='cuda:0',cache_path='./esm_cache') as esm:
    predictor = thermalFold_predictor(cfg_fname=cfg_fname,weight_path=weight_path,esm=esm,device='cuda:0')
    for name,seq in tqdm(zip(df['name'],df['sequence']),total=len(df)):
        tpath = Path(f"ThermalFold_prediction/{name}")
        tpath.mkdir(exist_ok=True,parents=True)
        for temp in temp_lst:
            tfname = tpath/f"NAME={name}:TEMP={temp}.pdb"
            if tfname.is_file(): continue
            inputs = [[seq,temp]]
            res = predictor.predict(inputs,num_samples=1)[0]
            with open(tfname,'w') as f:f.write(res)
torch.cuda.empty_cache()

Seed set to 42
Using cache found in /home/b208/.cache/torch/hub/facebookresearch_esm_main
100%|██████████| 3/3 [01:23<00:00, 27.97s/it]


In [ ]:
# Install the py3Dmol to visualize the predicted structures
!pip install py3Dmol

In [9]:
import py3Dmol

fname = Path("ThermalFold_prediction/Ice_structuring_protein/NAME=Ice_structuring_protein:TEMP=363.pdb")
with open(fname, 'r') as f:
    pdb_str = f.read()


view = py3Dmol.view(width=600, height=500)
view.addModel(pdb_str, 'pdb')


view.setStyle({'ss': 'h'}, {'cartoon': {'color': '#c23779'}})
view.setStyle({'ss': 's'}, {'cartoon': {'color': '#f7b731'}})
view.setStyle({'ss': 'c'}, {'cartoon': {'color': 'grey'}}) 

view.zoomTo()
view

3Dmol.js failed to load for some reason. Please check your browser console for error messages.